# Scanpy Tutorial
BroadRATs Talk: 6/8/2022  
Written and given by Danielle Firer

Tutorial based off of the Scanpy tutorial: Preprocessing and clustering 3k PBMCs
https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html
with some variation

In [ ]:
#### command line code to download the data and unpack it ####
mkdir data
wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
mkdir write

In [ ]:
#begin by importing necessary packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, figsize= (4,4), facecolor='white', fontsize=19)

In [ ]:
adata = sc.read_10x_mtx('/mnt/disk2/filtered_gene_bc_matrices/hg19', var_names='gene_symbols', cache=True)

In [ ]:
print(adata)
print('\nadata.var')
display(adata.var)
print('\nadata.obs')
display(adata.obs)
print('\nadata.X')
display(adata.X)

## Preprocessing

### Annotate whether genes are mitochondrial and calculate QC metrics for cells

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata.obs

In [ ]:
plt.scatter(adata.obs.total_counts, adata.obs.n_genes_by_counts, cmap='viridis' ,c=adata.obs['pct_counts_mt'],s=1)
plt.ylabel('n_genes')
plt.xlabel('total_counts')
plt.colorbar()

10X is only showing us their perfect datasets... 
More realistic plot of # counts vs # genes
![example_counts_vs_genes](example_counts_vs_genes.png)


A violin plot of some of the computed quality measures:

* the number of genes expressed in the count matrix
* the total counts per cell
* the percentage of counts in mitochondrial genes

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

Incorporating scanpy plots into subplots

In [ ]:
fig, axes = plt.subplots(1,4,figsize=(16,4))
t1 = sc.pl.violin(adata, 'total_counts', groupby=None, cut=0, ax=axes[0], show=False) #log=True,
t2 = sc.pl.violin(adata, 'pct_counts_mt', groupby=None, ax=axes[1], show=False)
sns.distplot(adata.obs['total_counts'], kde=False, bins=200, ax=axes[2])
axes[2].axvline(3000)
axes[2].set_yscale('log')
axes[3]= plt.scatter(adata.obs.total_counts, adata.obs.n_genes_by_counts, cmap='viridis' ,c=adata.obs['pct_counts_mt'],s=1)
plt.show()

### Basic filtering 

filter based on the number of genes found in a cell and the number of cells genes are found in

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

Mitochondrial filtering by slicing the AnnData object

If you were running doublet detection tools like scrublet, you would run them before removing cells based on mitochondrial QC metrics

In [ ]:
adata = adata[adata.obs.pct_counts_mt < 5, :] # 5% is very low, often times will use a higher cutoff ~10-20% but this depends on data quality
adata

### Total-count normalize, logarithmize, identify highly-variable genes and scale the data

In [ ]:
adata.layers['counts']= adata.X.copy() #save a copy of the raw counts in a layer
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata.copy() #save the log-norm counts before scaling

sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pp.scale(adata, max_value=10)

## PCA, computing the neighborhood graph, and clustering

In [ ]:
sc.pp.pca(adata, svd_solver='arpack', use_highly_variable=True)
sc.pl.pca_variance_ratio(adata, log=True)

What we see above is the elbow plot: we need to choose how many PCs to use

* the general rule is you want to include anything to the left of the elbow
* usually choose a PC the is just after the elbow
* if you choose too many PCs your UMAP will have too many very tiny clusters where you can't see any informative separation

An example of a UMAP after choosing too many PCs:  
![example_too_many_pcs](too_many_pcs.png)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=8)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])

In [ ]:
sc.tl.louvain(adata, resolution=.6) #lower resolution -> less clusters

In [ ]:
sc.pl.umap(adata, color=['louvain', 'CST3', 'NKG7'])

### changing UMAP colors

In [ ]:
adata

In [ ]:
adata.uns['louvain_colors']

In [ ]:
adata.uns['louvain_colors']= ['r','b','k','c','y','lightgrey','g','m','tab:orange','tab:brown']
sc.pl.umap(adata, color=['louvain'])

### Finding marker genes

Find marker genes that separate each louvain cluster vs the rest of the cells

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
adata.uns['rank_genes_groups']

Compare a cluster vs another cluster rather than all other cells:

example: have two tumor clusters and want to see why there is separation between the two of them

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain', groups=['0'], reference='5', method='wilcoxon')
sc.pl.rank_genes_groups(adata, groups=['0'], n_genes=20)

In [ ]:
#more detailed view for a certain group
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)

In [ ]:
sc.pl.violin(adata, ['CST3', 'NKG7', 'PPBP'], groupby='louvain')

In [ ]:
sc.pl.umap(adata, color= ['CST3', 'FCER1A','MS4A7','NKG7','GNLY','CD8A','MS4A1','LYZ', 'IL7R', 'CD14','PPBP','louvain'], ncols=5)

In [ ]:
sc.pl.violin(adata, ['CST3', 'FCER1A','MS4A7','NKG7','GNLY','CD8A','MS4A1','LYZ', 'CD14','IL7R','PPBP'], groupby='louvain')

                                                            Based on expression patterns:

| Louvain Group | Markers     | Cell Type
| ------------- | ----------- | ----------- 
| 0             | IL7R       | CD4 T cells
| 1             | IL7R       | CD4 T cells
| 2             | CD14, LYZ        | CD14+ Monocytes
| 3             | MS4A1   | B cells
| 4             | CD8A | CD8 T cells
| 5             | FCGR3A, MS4A7 | FCGR3A+ Monocytes
| 6             | GNLY, NKG7 | NK cells
| 7             | IL7R       | CD4 T cells
| 8             | FCER1A, CST3 | Dendritic Cells
| 9             | PPBP | Megakaryocytes



In [ ]:
cell_type_annotations = {'CD4 T': [0,1,7], 'CD14 Monocytes': [2], 'B': [3], 'CD8 T': [4], 'NK': [6], 'FCGR3A Monocytes': [5], 'Dendritic': [8], 'Megakaryocytes': [9]}

cluster_annotations = {}
for k,v in cell_type_annotations.items():
    for clus in v:
        cluster_annotations[clus]= k
cluster_annotations

In [ ]:
set(adata.obs.louvain)

In [ ]:
adata.obs['cell_type']= adata.obs.louvain.apply(lambda x: cluster_annotations[int(x)])
adata.obs['cell_type']

In [ ]:
sc.pl.umap(adata, color='cell_type', legend_loc='on data', title='', frameon=False, legend_fontsize=10)


In [ ]:
marker_genes = {     'NK': ['GNLY', 'NKG7'],
                     'CD8 T-cell': ['CD8A'],
                     'CD4 T-cell': ['IL7R'],
                     'B-cell': ['MS4A1'],
                     'CD14+ Monocytes': ['CD14', 'LYZ'],
                     'FCGR3A+ Monocytes': ['FCGR3A','MS4A7'],
                     'Megakaryocytes': ['PPBP'],
                     'Dendritic': ['FCER1A','CST3']}

sc.pl.dotplot(adata, marker_genes, groupby='louvain')

In [ ]:
sc.pl.stacked_violin(adata, marker_genes, groupby='louvain', rotation=90)

## Write the anndata object to a h5 file so that you can save, reload your progress and keep working later

In [ ]:
adata.write_h5ad('scanpy_tutorial.h5ad')

## if you need to grab the normalized counts and convert them to a dataframe

In [ ]:
log_norm_counts= pd.DataFrame.sparse.from_spmatrix(adata.raw.X)
log_norm_counts.columns= list(adata.var_names)
log_norm_counts.index= list(adata.obs_names)